# **Gradient SDK Tutorial**

In [ ]:
!pip install --pre -U gradient

In [ ]:
!pip install jupyter-tensorboard
!pip install -q tf-nightly-2.0-preview
!pip install -U setuptools

In [ ]:
#Import the SdkClient from the gradient package
from gradient import sdk_client, ProjectsClient

In [ ]:
#create a API Key
api_key = "api_key_here"

In [ ]:
deployment_client = sdk_client.DeploymentsClient(api_key)
models_client = sdk_client.ModelsClient(api_key)
jobs_client = sdk_client.JobsClient(api_key)
projects_client = ProjectsClient(api_key)
experiment_client = sdk_client.ExperimentsClient(api_key)

#or access them all from a single client
#client = sdk_client.SdkClient(api_key)

In [ ]:
#Reuse existing example, or create new project
existing_team = False
projects_list = projects_client.list()

#print out all projects in the team
for project in projects_list:
    if project.name == "SDK Example":
        print(project)
        existing_team = True
        break
    elif not existing_team:
        print(project)


help(projects_client.list)

In [ ]:
#get id of existing project or create a new one
proj_id = project.id if existing_team else projects_client.create("SDK Example")
print(proj_id)

help(projects_client.create)

In [ ]:
#create experiment client
experiment_client = sdk_client.ExperimentsClient(api_key)

#list experiments in the project
print(experiment_client.list(project_id = proj_id))
help(experiment_client.list)

In [ ]:
single_node_path = "/storage/models/sdk_test"

env_variable = {
    "EPOCHS_EVAL":2,
    "TRAIN_EPOCHS":3,
    "MAX_STEPS":100,
    "EVAL_SECS":10
}

single_node_parameters = {
    "name" : "single_node_experiment-sdk",
    "project_id" : proj_id,
    "command" : "python mnist.py",
    "machine_type" : "K80",
    "experiment_env": env_variable,
    "container": "tensorflow/tensorflow:1.13.1-gpu-py3",
    "workspace_url": "https://github.com/Paperspace/mnist-sample.git",
    "model_type": "Tensorflow",
    "model_path": single_node_path
}

In [ ]:
#create single node experiment
exp_id = experiment_client.create_single_node(**single_node_parameters)
print(exp_id)

#get single node experiment object
from gradient import constants
#experiment state, created but not started
state = experiment_client.get(exp_id).state
print("state: "+constants.ExperimentState.get_state_str(state))

print()
print()

help(experiment_client.create_single_node)

In [ ]:
#get experiment object
print(experiment_client.get(exp_id))

In [ ]:
#start experiment
resp = experiment_client.start(exp_id)
print(resp)

In [ ]:
#create a log stream & print all logs for the duration of experiment
print("Watching state of experiment")
state = "created"
while state != "running" or state != "stopped":
    new_state = constants.ExperimentState.get_state_str(experiment_client.get(exp_id).state)
    if new_state != state:
        print("state: "+state + "  new state: "+new_state)
        state = new_state
    if state == "running": break

log_stream = experiment_client.yield_logs(exp_id)
print("Streaming logs of experiment")
try:
    while True:
        print(log_stream.send(None))
except:
    print("done streaming logs")

In [ ]:
models_client = sdk_client.ModelsClient(api_key)

In [ ]:
#get model we just trained
model_sn = models_client.list(experiment_id = exp_id)[0]

#print model summary
print(model_sn.summary)

In [ ]:
#view experiment in tensorboard
%load_ext tensorboard
%tensorboard --logdir single_node_path

In [ ]:
##Create a dictionary of parameters for running a distributed/multinode experiment
env = {
    "EPOCHS_EVAL":5,
    "TRAIN_EPOCHS":10,
    "MAX_STEPS":1000,
    "EVAL_SECS":10
}

multi_node_parameters = { 
    "name": "multiEx",
    "project_id": proj_id,
    "experiment_type_id": 2,
    "worker_container": "tensorflow/tensorflow:1.13.1-gpu-py3",
    "worker_machine_type": "K80",
    "worker_command": "pip install -r requirements.txt && python mnist.py",
    "experiment_env": env,
    "worker_count": 2,
    "parameter_server_container": "tensorflow/tensorflow:1.13.1-gpu-py3",
    "parameter_server_machine_type": "K80",
    "parameter_server_command": "pip install -r requirements.txt && python mnist.py",
    "parameter_server_count": 1,
    "workspace_url": "https://github.com/Paperspace/mnist-sample.git",
    "model_type": "Tensorflow"
}

In [ ]:
mn_exp_id = experiment_client.create_multi_node(**multi_node_parameters)

In [ ]:
experiment_client.start(mn_exp_id)

In [ ]:
from gradient import constants
#create a log stream & print all logs for the duration of experiment
print("Watching state of experiment")
state = "created"
while state != "running" or state != "stopped":
    new_state = constants.ExperimentState.get_state_str(experiment_client.get(mn_exp_id).state)
    if new_state != state:
        print("state: "+state)
        state = new_state
    if state == "running": break

log_stream = experiment_client.yield_logs(mn_exp_id)
print("Streaming logs of experiment")
try:
    while True:
        print(log_stream.send(None))
except:
    print("done streaming logs")

# Run a tensorboard

In [ ]:
#view experiment in tensorboard
%load_ext tensorboard
%tensorboard --logdir "/storage/models/"+mn_exp_id

In [ ]:
model_mn = models_client.list(experiment_id = mn_exp_id)
print(model_mn[0].summary)

In [ ]:
model_to_deploy = None
model_to_deploy = mn_exp_id if model_mn[0]['accuracy']['result']['max'] > model_sn['accuracy']['result']['max'] else sn_exp_id

In [ ]:
model_deploy_id = models_client.list(experiment_id = model_to_deploy)[0].model_id

deploy_param = {
    "deployment_type" : "Tensorflow Serving on K8s",
    "image_url": "tensorflow/serving:latest-gpu",
    "name": "sdk_tutorial",
    "machine_type": "K80",
    "instance_count": 2,
    "model_id" : model_deploy_id
}

In [ ]:
resp = deployment_client.create(**deploy_param)

help(deployment_client.create)